In [1]:
from coir.data_loader import get_tasks
from coir.evaluation import COIR
from coir.models import YourCustomDEModel
from datasets import load_dataset, Dataset, concatenate_datasets
import pandas as pd
from query_expansion import expand_queries_and_qrels
from functions import * 
from collections import defaultdict

/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/envs/virtual-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


in tasks 
fetched data from hf
data loader init


20604it [00:00, 121574.20it/s]


In [2]:
cosqa_expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/Cosqa/CoSQA_explanations_query_code.csv'
cosqa_deepseek_updated_path='/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/postprocessing/output/COSQA_deepseek_explanations_clean.csv'
cosqa_granite_updated_path='/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/postprocessing/output/COSQA_granite_explanations_clean.csv'
csn_expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/explanations/CodeSearchNet_Python_deepseek_allSplits.csv'

#### Query Expansion

In [18]:
cosqa_deepseek_qe_path = '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/cosqa/cosqa_queries_expanded_deepseek_temp_0.csv'
cosqa_deepseek_qe_df = pd.read_csv(cosqa_deepseek_qe_path)
dataset_name = "cosqa"
tasks = load_data(dataset_name)
corpus, queries, qrels = tasks[dataset_name]
new_queries, new_qrels = expand_queries_and_qrels(cosqa_deepseek_qe_df, qrels)
tasks[dataset_name] = (corpus, new_queries, new_qrels)

in tasks 
fetched data from hf
data loader init


20604it [00:00, 217904.60it/s]


In [19]:
for qid, query in new_queries.items():
    if isinstance(query, float) and pd.isna(query):
        print(qid, 'nan')
        continue

#### Running the pipeline

In [3]:
dataset_name = "CodeSearchNet-python"
tasks = load_data(dataset_name)
corpus, queries, qrels = tasks[dataset_name]

in tasks 
fetched data from hf


Casting the dataset: 100%|██████████| 280652/280652 [00:00<00:00, 1371451.11 examples/s]


data loader init


280310it [00:04, 68029.15it/s]


In [6]:
corpus['c0']

{'text': 'def settext(self, text, cls=\'current\'):\n        """Set the text for this element.\n\n        Arguments:\n            text (str): The text\n            cls (str): The class of the text, defaults to ``current`` (leave this unless you know what you are doing). There may be only one text content element of each class associated with the element.\n        """\n        self.replace(TextContent, value=text, cls=cls)',
 'title': ''}

In [7]:
queries['q265734']

'str->list\n    Convert XML to URL List.\n    From Biligrab.'

In [21]:
# df= pd.read_csv(cosqa_deepseek_updated_path)
# corpus = update_corpus_with_cleaned_code(corpus, df) # updating the corpus with no comments in code
# #tasks[dataset_name] = (corpus, queries, qrels)
# tasks[dataset_name] = (corpus, new_queries, new_qrels)

In [22]:
#add_expl(tasks, "cosqa", cosqa_deepseek_updated_path, "explanation_deepseek_1_cleaned") # adding explanation instead of code

In [23]:
run("intfloat/e5-base-v2", tasks, "baseline", "dres", "cosqa_qe")

YourCustomDEModel init
COIR init!
in evaluation.py: loading up dres

in evaluation.py: retrieving

in beir/retrieval/evaluation.py: loading up search

in exact_search.py
len of queries:  89817


Encoding batches: 100%|██████████| 351/351 [02:00<00:00,  2.91batch/s]


len of corpus:  20604


Encoding batches: 100%|██████████| 81/81 [00:17<00:00,  4.71batch/s]


in evaluation.py: evaluating

Grouped evaluation based on query prefixes...

Ignoring identical query and document IDs...
Retrieval evaluation results saved to results/cosqa_qe/baseline/dres/intfloat/e5-base-v2
{'cosqa': {'NDCG': {'NDCG@1': 0.1584, 'NDCG@3': 0.18625, 'NDCG@5': 0.20362, 'NDCG@10': 0.2354, 'NDCG@100': 0.34644, 'NDCG@1000': 0.4001}, 'MAP': {'MAP@1': 0.05402, 'MAP@3': 0.11, 'MAP@5': 0.14931, 'MAP@10': 0.20879, 'MAP@100': 0.26167, 'MAP@1000': 0.26584}, 'Recall': {'Recall@1': 0.05402, 'Recall@3': 0.12237, 'Recall@5': 0.17659, 'Recall@10': 0.27323, 'Recall@100': 0.57733, 'Recall@1000': 0.81303}, 'Precision': {'P@1': 0.1584, 'P@3': 0.16643, 'P@5': 0.16681, 'P@10': 0.15275, 'P@100': 0.03759, 'P@1000': 0.00542}}}


#### duplicates

In [ ]:
duplicate_code_groups = df.groupby('code').filter(lambda x: len(x) > 1)
num_duplicate_code_groups = duplicate_code_groups['code'].nunique()
print(f"Number of code groups with >1 item: {num_duplicate_code_groups} out of {len(df)}")

In [24]:
duplicate_code_df = df.groupby('code').filter(lambda x: len(x) > 1)

result = (
    duplicate_code_df
    .groupby('code')
    .agg({
        'corpus_id': lambda x: list(set(x)),
        'query_id': lambda x: list(set(x))
    })
    .reset_index()
    .rename(columns={'corpus_id': 'corpus_ids_with_same_code', 'query_id': 'query_ids_for_those_corpus'})
)

print(result.head())


                                                code  \
0  async def _send_plain_text(self, request: Requ...   
1  async def _thread_coro(self, *args):\n        ...   
2  async def async_input(prompt):\n    """\n    P...   
3  async def async_run(self) -> None:\n        ""...   
4  async def cursor(self) -> Cursor:\n        """...   

                           corpus_ids_with_same_code  \
0              [d3961, d1874, d20448, d6227, d10337]   
1  [d6144, d118, d5150, d12781, d7622, d1889, d82...   
2  [d2781, d16258, d16413, d4944, d9133, d9137, d...   
3           [d19465, d20020, d16870, d19314, d19313]   
4                   [d16659, d18387, d19386, d17979]   

                          query_ids_for_those_corpus  
0              [q20448, q1874, q3961, q6227, q10337]  
1  [q11702, q5150, q6147, q7622, q8299, q118, q18...  
2  [q4944, q16413, q9137, q16258, q20250, q9133, ...  
3           [q20020, q19314, q16870, q19465, q19313]  
4                   [q17979, q19386, q16659, q18387

In [17]:
duplicate_code_groups = df.groupby('cleaned_code').filter(lambda x: len(x) > 1)

num_duplicate_code_groups = duplicate_code_groups['cleaned_code'].nunique()

print(f"Number of cleaned_code groups with >1 item: {num_duplicate_code_groups} out of {len(df)}")

Number of cleaned_code groups with >1 item: 3406 out of 20604


In [16]:
duplicate_code_groups = df.groupby('explanation_deepseek_1_cleaned').filter(lambda x: len(x) > 1)

num_duplicate_code_groups = duplicate_code_groups['explanation_deepseek_1_cleaned'].nunique()

print(f"Number of explanation_deepseek_1_cleaned groups with >1 item: {num_duplicate_code_groups} out of {len(df)}")

Number of explanation_deepseek_1_cleaned groups with >1 item: 93 out of 20604
